In [1]:
import os
import subprocess
import base64
import json
import xml.etree.ElementTree as ET
import datetime
import requests
import re

# Generate Prompt

In [2]:
def flatten_dict(d, parent_key=""):
    """
    Recursively flattens a nested dict into { "full.path": "string", ... }.
    If the value is a dict, recurse; if it's a string, store it.
    """
    flattened = {}
    for key, value in d.items():
        if parent_key:
            full_key = f"{parent_key}.{key}"
        else:
            full_key = key

        if isinstance(value, dict):
            flattened.update(flatten_dict(value, full_key))
        elif isinstance(value, str):
            flattened[full_key] = value
        else:
            # If list/number/None => skip or handle as needed
            pass
    return flattened

In [3]:
prompts = []
with open('elements.json', 'r', encoding='utf-8') as f:
    prompts = flatten_dict(json.load(f))
prompts

{'Verarbeitungskennzeichen': "Suche nach dem Verarbeitungskennzeichen, das meistens mit einem Buchstaben angegeben ist, z.B. 'N'.",
 'PosNrAuftragCSTransport': "Suche nach einer Nummer, die als Positionsnummer für den CS Transport bezeichnet ist, z.B. '2301231'.",
 'Auftragsart': "Finde die Art des Auftrags, häufig durch eine Abkürzung wie 'KL' gekennzeichnet.",
 'ReferenznummerAuftrag': "Suche nach der Referenznummer des Auftrags, die oft durch 'Nr.' gefolgt von einer Nummer/Kombination angegeben wird, z.B. '32231423 / 21.5640'.",
 'RelationAuftrag': 'Finde den Teil des Dokuments, der eine spezifische Relation für den Auftrag angibt. Diese Information kann möglicherweise fehlen.',
 'Auftraggeber': "Suche nach der E-Mail-Adresse des Auftraggebers, oft als Kontaktinformation angegeben, z.B. 'f.baumann@christof-group.com'.",
 'KontaktAuftraggeber': "Finde den Namen oder die Bezeichnung der kontaktierenden Person für den Auftrag, z.B. 'User 4 (TR)'.",
 'Rechnungsempfaenger': "Suche nach d

In [4]:
flattened_prompt = "\n".join([f"{k}: {v}" for k,v in prompts.items()])

# Contact Antropic

In [5]:
def pack_file(file_path, file_type, mime_type):
    with open(file_path, "rb") as pdf_file:
        file_contents = pdf_file.read()
        return {
            "type": "document",
            "source": {
              "type": "base64",
              "media_type": mime_type,
              "data": base64.b64encode(file_contents).decode("utf-8"),
            }
        }

def pack_pdf(pdf_file):
    return pack_file(pdf_file, "document", "application/pdf")

def pack_png(png_file):
    return pack_file(png_file, "image", "image/png")


def pack_text(text):
    return {
        "type": "text",
        "text": text
    }

In [6]:
ANTHROPIC_API_KEY=gpt_api_key=open('.anthropic_key', 'r').read().replace('\n','')

In [22]:
# Define the API endpoint and headers
url = "https://api.anthropic.com/v1/messages"
headers = {
    "content-type": "application/json",
    "x-api-key": ANTHROPIC_API_KEY,
    "anthropic-version": "2023-06-01",
}

In [38]:
def call_anthropic(messages):
    data = {
        "model": "claude-sonnet-4-0",
        "max_tokens": 8192,
        "temperature": 0,
        "messages": messages,
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code != 200:
        raise Exception(response.text)
    content = response.json()['content']
    print("LLM Usage:", json.dumps(response.json()['usage']))
    #data['messages'].append({"role":"assistant", "content":content})
    return content

# Full extraction

In [39]:
input_files=[file for file in os.listdir("input") if re.match(r".*pdf", file)]
input_files

['Auftrag Scheucher1762.pdf',
 'Auftrag Soprema3.pdf',
 'Auftrag Scheucher4053.pdf',
 'Auftrag Soprema1.pdf',
 'Auftrag Scheucher5000.pdf',
 'Auftrag Scheucher3063.pdf',
 'Auftrag ACE1.pdf',
 'Auftrag Haberkorn21.pdf',
 'Auftrag ACE2.pdf',
 'Auftrag Haberkorn22.pdf',
 'Auftrag NUN5.pdf',
 'Auftrag Scheucher1023.pdf',
 'Auftrag NUN2.pdf',
 'Auftrag NUN3.pdf',
 'Auftrag NUN1.pdf']

In [40]:
def extract_json(text):
    text = re.sub("^[^{]*","", text)
    text = re.sub("[^}]*$","", text)
    return json.loads(text)

In [41]:
def process_file(input_file):
    messages_content = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Du bist ein OCR system zum Erkennen strukturierter Daten in Dokumenten. Du erkennst "
                            "Aufträge für die Firma ant-sfer Speditions GmbH. antsfer.com ist also der Auftragnehmer."
                            " Der Absender des Auftrages ist der Auftraggeber, niemals ans-fer selber. Die Aufträge kommen als "
                            "Dokumente vom Kunden und sollen in das CRM von ant-sfer Speditions GmbH eingepflegt "
                            "werden. Die Aufträge können manuell überarbeitet sein: Durchgestrichene Absätze "
                            "bitte nicht erkennen."
                }, {
                    "type": "text",
                    "text": "Gib alle Längenangaben in cm an, außer wenn im Namen des Feldes explizit meter steht. "
                            "Zahlen sollen mit dem deutschen Komma formatiert werden. "
                            "Falls nur eine Entladestation angegeben ist, dann soll die Beladestation mit den Daten des Auftaggebers gefüllt werden."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                pack_text(f"Erkenne die Daten in den Bildern. Gib Deinen Output als JSON aus und fülle die "
                            f"folgenden Elemente mit Werten: \n"
                            f"Wenn Du einen wert nicht finden "
                            f"kannst, liefere \"\" als Wert zurück."),
                pack_text(flattened_prompt),
                pack_pdf(input_file)
            ]
        }
    ]
    try:
        response = call_anthropic(messages_content)
        as_json= extract_json(response[0]['text'])
        as_json= flatten_dict(as_json)
        return as_json
    except Exception as e:
        raise Exception(f"Error processing {input_file}") from e 

In [42]:
input_file= "input/Auftrag Scheucher1762.pdf"
messages_content = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Du bist ein OCR system zum Erkennen strukturierter Daten in Dokumenten. Du erkennst "
                        "Aufträge für die Firma ant-sfer Speditions GmbH. antsfer.com ist also der Auftragnehmer."
                        " Der Absender des Auftrages ist der Auftraggeber, niemals ans-fer selber. Die Aufträge kommen als "
                        "Dokumente vom Kunden und sollen in das CRM von ant-sfer Speditions GmbH eingepflegt "
                        "werden. Die Aufträge können manuell überarbeitet sein: Durchgestrichene Absätze "
                        "bitte nicht erkennen."
            }, {
                "type": "text",
                "text": "Gib alle Längenangaben in cm an, außer wenn im Namen des Feldes explizit meter steht. "
                        "Zahlen sollen mit dem deutschen Komma formatiert werden. "
                        "Falls nur eine Entladestation angegeben ist, dann soll die Beladestation mit den Daten des Auftaggebers gefüllt werden."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            pack_text(f"Erkenne die Daten in den Bildern. Gib Deinen Output als JSON aus und fülle die "
                        f"folgenden Elemente mit Werten: \n"
                        f"Wenn Du einen wert nicht finden "
                        f"kannst, liefere \"\" als Wert zurück."),
            pack_text(flattened_prompt),
            pack_pdf(input_file)
        ]
    }
]

In [43]:
results = [process_file("input/" + input_file) for input_file in input_files]

LLM Usage: {"input_tokens": 6100, "cache_creation_input_tokens": 0, "cache_read_input_tokens": 0, "output_tokens": 1255, "service_tier": "standard"}
LLM Usage: {"input_tokens": 10084, "cache_creation_input_tokens": 0, "cache_read_input_tokens": 0, "output_tokens": 1270, "service_tier": "standard"}
LLM Usage: {"input_tokens": 6092, "cache_creation_input_tokens": 0, "cache_read_input_tokens": 0, "output_tokens": 1258, "service_tier": "standard"}
LLM Usage: {"input_tokens": 5879, "cache_creation_input_tokens": 0, "cache_read_input_tokens": 0, "output_tokens": 1343, "service_tier": "standard"}
LLM Usage: {"input_tokens": 6098, "cache_creation_input_tokens": 0, "cache_read_input_tokens": 0, "output_tokens": 1253, "service_tier": "standard"}
LLM Usage: {"input_tokens": 6095, "cache_creation_input_tokens": 0, "cache_read_input_tokens": 0, "output_tokens": 1250, "service_tier": "standard"}
LLM Usage: {"input_tokens": 10084, "cache_creation_input_tokens": 0, "cache_read_input_tokens": 0, "outpu

In [46]:
for file_name, result in zip(input_files, results):
    with open("output/"+ re.sub(r"pdf$", "json", file_name), 'w') as file:
        json.dump(result, file)